# Figure 1
## Methods
* wget
* BLAST
* BioAlignments.jl
* WASITAMATchISAW.jl

### a) Download Swiss-Prot and SCOP2
* Download Swiss-Prot
    * https://www.uniprot.org/help/downloads
    * unzip the .gz archive.
* Download SCOP2 superfamily representatives
    * https://scop.mrc-lmb.cam.ac.uk/download
    * right-click and "Save Page as..."
    
Both files should be in the FASTA format, e.g.:

`/home/georgeglidden/Downloads/scop_sf_represeq_lib_latest.fa`

`/home/georgeglidden/Downloads/uniprot_sprot.fasta`

Create a data folder for the FASTA files, e.g.:

`cd /home/georgeglidden/Desktop/WASITAMATchISAW/`

`mkdir data/`

`mv /home/georgeglidden/Downloads/scop_sf_represent_lib_latest.fa data/scop2sf.fa`

`mv /home/georgeglidden/Downloads/uniprot_sprot.fasta data/sprot.fa`

A subdirectory for the BLAST database will also be necessary:

`mkdir data/scop2sfdb`

Eventually, the outputs directory will also be needed:

`mkdir outputs`

### b) Register Swiss-Prot to SCOP2 superfamily representatives
Make a database for SCOP2 

`makeblastdb -in data/scop2sf.fa -db data/scop2sfdb/scop2sf -dbtype prot -out data/scop2sfdb/scop2sf -title data/scop2sfdb/scop2sf`

Collect alignments between Swiss-Prot and SCOP2

`blastp -query data/sprot.fa /data/scop2sfdb/scop2sf -word_size 4 -outfmt 7 -num_threads 12 > outputs/sprot_scop2.txt`

In [ ]:
# locate datasets
pathtoscop2sf = "data/scop2sf.fa"
pathtosprot = "data/sprot.fa"
# make a database for SCOP2.
pathtoscop2sfdb = "data/scop2sfdb/scop2sf"
include("source/blast.jl")
using .BLAST
BLAST.makeblastdb(pathtoscop2sf, pathtoscop2sfdb)

In [ ]:
# collect alignments between Swiss-Prot and SCOP2.
# THIS CELL MAY TAKE SEVERAL HOURS TO COMPUTE
sprot_scop2sf_alignment = BLAST.blastp(pathtosprot, pathtoscop2sfdb, numthreads=12)

In [ ]:
# clean up the database
BLAST.cleanup_makeblastdb(pathtoscop2sfdb)
# parse BLAST output table to a DataFrame
# write to outputs/

### c) Restrict Swiss-Prot by SCOP2 superfamily pctid

In [ ]:
# load Swiss-Prot / SCOP2 alignment output into a DataFrame

In [8]:
# calculate percent ID for each alignment
# restrict Swiss-Prot to the sequences with greater than 95% percent identity to a SCOP2 superfamily representative sequence

### d) Plot lengths distribution 

In [10]:
# calculate lengths distribution for Swiss-Prot and Swiss-Prot |{>=95} SCOP2
# save to CSVs